In [4]:
#!pip install openpyxl --proxy=http://internet.ford.com:83

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 2.0 MB/s eta 0:00:0000:0100:01


In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./output/topics_cpr_Aug.csv')  
data

,Unnamed: 0,id,text,month,year,text_n,Sub_category,Category
0,0,R_1QXxniNV3YpqUPq,It was about new vehicle,Aug,2024,It was about new vehicle.,New Vehicles,Product related
1,1,R_3DhVOZ3PTKZkWgp,Yes I have seen commercials for ford,Aug,2024,Yes I have seen commercials for ford.,Other,Other
2,2,R_5Ea54NwgY2tkvR6,buy a ford product,Aug,2024,buy a ford product.,Other,Other
3,3,R_1YEbWB9AKWFLJnK,largest employer in usa,Aug,2024,largest employer in usa.,Other,Other
4,4,R_1co47Ec7FvuwRpO,Fords recall of F-150 vehicles,Aug,2024,Fords recall of F-150 vehicles.,Recalls,Product related
...,...,...,...,...,...,...,...,...
467,467,R_6nCY5c1j1ZGUX9S,that it was exapanding,Aug,2024,that it was exapanding.,Other,Other
468,468,R_3t9fVWP70zFU9iP,It has a good fit for motor that make a good f...,Aug,2024,It has a good fit for motor that make a good f...,Other,Other
469,469,R_1n0P0NNv5Ijvesp,Great,Aug,2024,Great.,Other,Other
470,470,R_3P3eupNFhLpNxFT,No trucks out,Aug,2024,No trucks out.,Other,Other


In [3]:
data['category_n'] = data['Sub_category'].str.split(', ')
#data['category_n'] = data['sentiment'].str.split(', ')
data_split = data.explode('category_n')
data_split = data_split.reset_index(drop=True)
data_split.head()

size = 20        # sample size
replace = False  # without replacement
ab = data_split.groupby(['category_n']).count()>=size
bc = ab[ab['id']==True].index
new_data = []
for i in range(0,len(bc)):
    sub=data_split[data_split['category_n']==bc[i]]
    new_data.append(sub)
new_data = pd.concat(new_data)    

fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
sub_data1 = new_data.groupby('category_n', as_index=False).apply(fn)
sub_data2 = data_split[data_split.index.isin(new_data.index) == False]
final = pd.concat([sub_data1,sub_data2])

In [4]:
final.to_csv('./accuracy_sample_Aug24_topic.csv')

## Test

In [6]:
df = pd.read_csv('./accuracy_sample_Aug24_topic_annotated.csv')
df.head(2)

,id,text,month,year,text_n,Sub_category,annotated_sub_category,Category,annotated_category
0,R_3Lxu2j5LG6EAjnj,How they plan on making a electric car one day,Aug,2024,How they plan on making a electric car one day.,Electric Vehicles,Electric Vehicles,Electrification,Electrification
1,R_6h5sHHiDmZ99CSZ,Ford new mustang that is all ev as well as oth...,Aug,2024,Ford new mustang that is all ev as well as oth...,Electric Vehicles,Electric Vehicles,Electrification,Electrification


In [7]:
df['Category'] = df['Category'].astype(str)
df['annotated_category'] = df['annotated_category'].astype(str)
topics_pred = df.loc[:,'Category']
topics_test = df.loc[:,'annotated_category']

from sklearn.metrics import classification_report
print(classification_report(topics_test, topics_pred))

from sklearn.metrics import accuracy_score
print("Accuracy Score -> ",accuracy_score(topics_pred, topics_test)*100)

                                             precision    recall  f1-score   support

                                   Branding       1.00      0.78      0.88        18
                           Corporate issues       1.00      1.00      1.00        13
                            Electrification       1.00      1.00      1.00        20
New technology, unique features or services       1.00      1.00      1.00         5
                      Organization feedback       1.00      1.00      1.00        25
                                      Other       0.65      1.00      0.79        13
                            Product related       1.00      0.95      0.98        63

                                   accuracy                           0.96       157
                                  macro avg       0.95      0.96      0.95       157
                               weighted avg       0.97      0.96      0.96       157

Accuracy Score ->  95.54140127388536


In [8]:
df['Sub_category'] = df['Sub_category'].astype(str)
df['annotated_sub_category'] = df['annotated_sub_category'].astype(str)
topics_pred = df.loc[:,'Sub_category']
topics_test = df.loc[:,'annotated_sub_category']

from sklearn.metrics import classification_report
print(classification_report(topics_test, topics_pred))

from sklearn.metrics import accuracy_score
print("Accuracy Score -> ",accuracy_score(topics_pred, topics_test)*100)

                                 precision    recall  f1-score   support

Brand Perception and Reputation       1.00      0.78      0.88        18
              Electric Vehicles       1.00      1.00      1.00        20
   Environmental Sustainability       1.00      1.00      1.00         5
          Financial Performance       1.00      1.00      1.00        20
      Innovation and Technology       1.00      1.00      1.00         5
                Labor Relations       1.00      1.00      1.00        13
                   New Vehicles       1.00      1.00      1.00        20
                          Other       0.65      1.00      0.79        13
Product Quality and Reliability       1.00      0.87      0.93        23
                        Recalls       1.00      1.00      1.00        20

                       accuracy                           0.96       157
                      macro avg       0.97      0.96      0.96       157
                   weighted avg       0.97      0